### LangChain Expression Language (LCEL)

In [2]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

### Simple Chain

In [3]:
prompt  = ChatPromptTemplate.from_template(
    "tell me short joke on the {topic}"
)

model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({'topic': "bears"})



C:\Users\Dinesh\AppData\Local\Temp\ipykernel_1168\1753728364.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


"Why don't bears like fast food? Because they can't catch it!"

### More Complex chain

In [6]:
# And Runnable Map to supply user-provided inputs to the prompt

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_texts(
    ['harrison worked at google', 'bears like to eat honey'],
    embedding= OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()



c:\Users\Dinesh\anaconda3\envs\ragenv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [7]:
retriever.get_relevant_documents("where did harrison work? ")

C:\Users\Dinesh\AppData\Local\Temp\ipykernel_1168\177785137.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrison work? ")


[Document(metadata={}, page_content='harrison worked at google'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [8]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at google')]

In [9]:
template = """Answer the following based on the following context: {context}
question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)


In [10]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question":lambda x: x['question']
}) | prompt | model | output_parser

In [11]:
chain.invoke({"question": "where did harrison work"})

'Harrison worked at Google.'

In [13]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x['question']),
    "question":lambda x: x['question']
})

inputs.invoke({"question": "where did harrison work"})

{'context': [Document(metadata={}, page_content='harrison worked at google'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work'}

## Bind

In [15]:
# and openai functions

functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [21]:
prompt = ChatPromptTemplate.from_messages (
    [
        ("human", "{input}")
    ]
)

model = ChatOpenAI(temperature= 0).bind(functions=functions)

In [24]:
import json

runnable = prompt | model
runnable.invoke({"input": "what is the weather in sf"})


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e41b75af-1541-4d1b-881c-a865fc8d0618-0')

In [25]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [26]:
model = model.bind(functions=functions)

runnable = prompt | model

runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b09239ae-5354-403a-907d-153658601b70-0')

### Fallbacks

In [28]:
from langchain.llms import OpenAI
import json

In [32]:
simple_model = OpenAI(
    temperature=0,
    max_tokens = 1000,
    model = "gpt-3.5-turbo-instruct"
)

simple_chain = simple_model | json.loads

In [33]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [34]:
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

In [ ]:
simple_chain.invoke(challenge)

"""The issue lies in the second step of your chain: you're using json.loads to parse the output of the OpenAI model, 
but the model's response might not be valid JSON or could include additional text/metadata outside the JSON structure. 
This is likely what's causing the JSONDecodeError. Here's how to address and debug the issue"""


JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [38]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [39]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [40]:
final_chain = simple_chain.with_fallbacks([chain])

final_chain.invoke(challenge)

{'poem1': {'title': 'The Night Sky',
  'author': 'Emily Dickinson',
  'firstLine': 'The night is starry and the stars are blue.'},
 'poem2': {'title': 'Autumn Leaves',
  'author': 'Robert Frost',
  'firstLine': "My sorrow, when she's here with me, thinks these dark days of autumn rain are beautiful as days can be."},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul.'}}

### Interface

In [41]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [42]:
chain.invoke({"topic":"bears"})

"Why don't bears like fast food? Because they can't catch it!"

In [43]:
chain.batch([{"topic":"bears"},{"topic":"frogs"}])

["Why did the bear break up with his girlfriend?\nHe couldn't bear the relationship anymore!",
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

In [44]:
for t in chain.stream({'topic':"frog"}):
    print(t)


Why
 did
 the
 frog
 go
 to
 the
 bank
?
 To
 hop
-p
en
 a
 savings
 account
!



In [45]:
response = await chain.ainvoke({'topic':"bears"})
response

'Why do bears have hairy coats?\n\nFur protection!'